# How to load the MGF parquet files and Sample Metadata from MinIO into duckdb

There are 7 summary analyses files for each MetaGOflow analysis result:

    "metagoflow_analyses.SSU", <- taxonomy using SSU
    "metagoflow_analyses.LSU", <- taxonomy using LSU
    "metagoflow_analyses.go_slim", <- functional annotations using Gene Ontology SLIM
    "metagoflow_analyses.go", <- functional annotation using Gene Ontology
    "metagoflow_analyses.ips", <- functional annotation using Inter Pro Scan
    "metagoflow_analyses.ko", <- functional analyses using KEGG Ontology
    "metagoflow_analyses.pfam" <- functional annotation using PFAM

Parquet tables have been created for each of these table types, one table from each of the 54 samples, chained on top of each other into a meta-table, with the addition of two further columns.

column[0] is the EMO-BON "ref-code" from the run-information-batch-00[X].csv files on github,
column[1] is a shortened version of the "reads_name" from the same file that identifies the raw DNA data associated with the sample and metagoflow analysis.

all the parquet tables have these two columns prepended.

In [5]:
import os
import io
import math
import duckdb # type: ignore
import numpy
import json
from duckdb import CatalogException, BinderException # type: ignore
import pandas as pd
from pandasql import sqldf
from minio import Minio, S3Error
from timeit_decorator import timeit

creds = json.load(open("credentials.json"))

client = Minio("10.4.1.4:9000",
    secure=False,
    access_key=creds["accessKey"],
    secret_key=creds["secretKey"],
) 

def get_object(bucket_name, file_format, file_name, verbose=True):
    if verbose:
        print(f"{bucket_name=} - {file_format=} - {file_name=}")
    try:
        response = client.get_object(bucket_name, file_name)
        buffer = io.BytesIO(response.read())
    except S3Error:
        raise
    finally:
        if file_format == "parquet":
            df = pd.read_parquet(buffer, engine='pyarrow')
        elif file_format == "csv":
            df = pd.read_csv(buffer)
        else:
            raise ValueError(f"Unknown {file_format=}")
        response.close()
        response.release_conn()
        if verbose:
            print(f"Downloaded {file_name} into dataframe")
    return df

### Get a list of current table versions available (note this will break the next code when new versions are available)

In [2]:
# The destination bucket and filename on the MinIO server
bucket_name = "emo-bon-tables"
for obj in client.list_objects(bucket_name):
    print(obj.object_name)
#Currently only v1 tables are available

v1/


### Load MGF and Sample Metadata into duckdb

In [3]:
# MGF parquet tables to dataframes
bucket_name = "emo-bon-tables"
objects = client.list_objects(bucket_name, recursive=True)
mgf_parquet_dfs = {}
for obj in objects:
    name = obj.object_name.split(".")[-2]
    df = get_object(bucket_name, "parquet", obj.object_name, verbose=False)
    mgf_parquet_dfs[name] = df

# Sample metadata
# Get the latest Batch combined logsheets file
# Remember we are downloading from MinIO
batch_file = "Batch1and2_combined_logsheets_2024-09-06.csv"
sample_metadata = ("emo-bon-metadata-tables", "csv", batch_file)
sample_metadata = get_object(*sample_metadata, verbose=False)

#Observatory metadata - from the GoogleSheets
observatory_metadata = ("emo-bon-metadata-tables", "csv", "Observatory_combined_logsheets_validated.csv")
observatory_metadata = get_object(*observatory_metadata, verbose=False)

# Into duckdb
try:
    duckdb.sql("DROP TABLE SAMPLE_METADATA")
    duckdb.sql("DROP TABLE OBS_METADATA")
    for table_name in mgf_parquet_dfs:
        cmd = f"DROP TABLE {table_name}"
        duckdb.sql(cmd)
except CatalogException:
    pass
duckdb.sql("CREATE TABLE SAMPLE_METADATA AS SELECT * FROM sample_metadata")
duckdb.sql("SELECT COUNT(*) FROM SAMPLE_METADATA")
duckdb.sql("CREATE TABLE OBS_METADATA AS SELECT * FROM observatory_metadata")
duckdb.sql("SELECT COUNT(*) FROM OBS_METADATA")
for table_name in mgf_parquet_dfs:
    df = mgf_parquet_dfs[table_name]
    cmd = f"CREATE TABLE {table_name} AS SELECT * FROM df"
    duckdb.sql(cmd)

duckdb.sql("SHOW TABLES")


┌─────────────────┐
│      name       │
│     varchar     │
├─────────────────┤
│ LSU             │
│ OBS_METADATA    │
│ SAMPLE_METADATA │
│ SSU             │
│ go              │
│ go_slim         │
│ ips             │
│ ko              │
│ pfam            │
└─────────────────┘

In [5]:
duckdb.sql("DESCRIBE SAMPLE_METADATA").show(max_rows=113)

┌─────────────────────────────┬─────────────┬─────────┬─────────┬─────────┬─────────┐
│         column_name         │ column_type │  null   │   key   │ default │  extra  │
│           varchar           │   varchar   │ varchar │ varchar │ varchar │ varchar │
├─────────────────────────────┼─────────────┼─────────┼─────────┼─────────┼─────────┤
│ source_mat_id               │ VARCHAR     │ YES     │ NULL    │ NULL    │ NULL    │
│ source_mat_id_orig          │ VARCHAR     │ YES     │ NULL    │ NULL    │ NULL    │
│ samp_description            │ VARCHAR     │ YES     │ NULL    │ NULL    │ NULL    │
│ tax_id                      │ BIGINT      │ YES     │ NULL    │ NULL    │ NULL    │
│ scientific_name             │ VARCHAR     │ YES     │ NULL    │ NULL    │ NULL    │
│ investigation_type          │ VARCHAR     │ YES     │ NULL    │ NULL    │ NULL    │
│ env_material                │ VARCHAR     │ YES     │ NULL    │ NULL    │ NULL    │
│ collection_date             │ VARCHAR     │ YES     

In [6]:
duckdb.sql("DESCRIBE OBS_METADATA").show(max_rows=25)

┌───────────────────────────────┬─────────────┬─────────┬─────────┬─────────┬─────────┐
│          column_name          │ column_type │  null   │   key   │ default │  extra  │
│            varchar            │   varchar   │ varchar │ varchar │ varchar │ varchar │
├───────────────────────────────┼─────────────┼─────────┼─────────┼─────────┼─────────┤
│ obs_id                        │ VARCHAR     │ YES     │ NULL    │ NULL    │ NULL    │
│ project_name                  │ VARCHAR     │ YES     │ NULL    │ NULL    │ NULL    │
│ latitude                      │ DOUBLE      │ YES     │ NULL    │ NULL    │ NULL    │
│ longitude                     │ DOUBLE      │ YES     │ NULL    │ NULL    │ NULL    │
│ geo_loc_name                  │ VARCHAR     │ YES     │ NULL    │ NULL    │ NULL    │
│ loc_broad_ocean               │ VARCHAR     │ YES     │ NULL    │ NULL    │ NULL    │
│ loc_broad_ocean_mrgid         │ BIGINT      │ YES     │ NULL    │ NULL    │ NULL    │
│ loc_regional                  

In [3]:
duckdb.sql("DESCRIBE LSU")

┌──────────────┬─────────────┬─────────┬─────────┬─────────┬─────────┐
│ column_name  │ column_type │  null   │   key   │ default │  extra  │
│   varchar    │   varchar   │ varchar │ varchar │ varchar │ varchar │
├──────────────┼─────────────┼─────────┼─────────┼─────────┼─────────┤
│ ref_code     │ VARCHAR     │ YES     │ NULL    │ NULL    │ NULL    │
│ reads_name   │ VARCHAR     │ YES     │ NULL    │ NULL    │ NULL    │
│ ncbi_taxid   │ BIGINT      │ YES     │ NULL    │ NULL    │ NULL    │
│ abundance    │ FLOAT       │ YES     │ NULL    │ NULL    │ NULL    │
│ superkingdom │ VARCHAR     │ YES     │ NULL    │ NULL    │ NULL    │
│ kingdom      │ VARCHAR     │ YES     │ NULL    │ NULL    │ NULL    │
│ phylum       │ VARCHAR     │ YES     │ NULL    │ NULL    │ NULL    │
│ class        │ VARCHAR     │ YES     │ NULL    │ NULL    │ NULL    │
│ order        │ VARCHAR     │ YES     │ NULL    │ NULL    │ NULL    │
│ family       │ VARCHAR     │ YES     │ NULL    │ NULL    │ NULL    │
│ genu

In [8]:
duckdb.sql("SELECT obs_id, loc_loc, contact_name FROM OBS_METADATA WHERE geo_loc_name = 'Portugal'")

┌──────────┬──────────────┬────────────────────┐
│  obs_id  │   loc_loc    │    contact_name    │
│ varchar  │   varchar    │      varchar       │
├──────────┼──────────────┼────────────────────┤
│ RFormosa │ Ria Formosa  │ Bruno Louro        │
│ RFormosa │ Ria Formosa  │ Bruno Louro        │
│ OSD74    │ Porto Valley │ Catarina Magalhães │
└──────────┴──────────────┴────────────────────┘

In [9]:
duckdb.sql("SELECT ref_code, abundance, reads_name, ncbi_taxid FROM LSU WHERE abundance > 10000 ORDER BY abundance DESC")

┌─────────────┬───────────┬──────────────────┬────────────┐
│  ref_code   │ abundance │    reads_name    │ ncbi_taxid │
│   varchar   │   float   │     varchar      │   int64    │
├─────────────┼───────────┼──────────────────┼────────────┤
│ EMOBON00125 │   14280.0 │ HVWGWDSX5.UDI133 │      54526 │
│ EMOBON00124 │   12106.0 │ HVWGWDSX5.UDI121 │      54526 │
│ EMOBON00121 │   11201.0 │ HVWGWDSX5.UDI109 │      54526 │
│ EMOBON00120 │   10968.0 │ HVWGWDSX5.UDI097 │      54526 │
│ EMOBON00127 │   10929.0 │ HVWGWDSX5.UDI105 │      72037 │
│ EMOBON00141 │   10292.0 │ HCFCYDSX5.UDI124 │       1236 │
└─────────────┴───────────┴──────────────────┴────────────┘

In [10]:
duckdb.sql("SELECT source_mat_id, ref_code, sea_surf_temp FROM SAMPLE_METADATA WHERE sea_surf_temp < 10.0")

┌─────────────────────────────────┬─────────────┬───────────────┐
│          source_mat_id          │  ref_code   │ sea_surf_temp │
│             varchar             │   varchar   │    double     │
├─────────────────────────────────┼─────────────┼───────────────┤
│ EMOBON_ESC68N_Wa_211104_3um_1   │ EMOBON00196 │          7.55 │
│ EMOBON_ESC68N_Wa_211104_3um_2   │ EMOBON00197 │          7.55 │
│ EMOBON_ESC68N_Wa_211104_0.2um_1 │ EMOBON00198 │          7.55 │
│ EMOBON_ESC68N_Wa_211104_0.2um_2 │ EMOBON00199 │          7.55 │
│ EMOBON_ESC68N_Wa_211215_3um_1   │ EMOBON00202 │          7.58 │
│ EMOBON_ESC68N_Wa_211215_3um_2   │ EMOBON00203 │          7.58 │
│ EMOBON_ESC68N_Wa_211215_0.2um_1 │ EMOBON00204 │          7.58 │
│ EMOBON_ESC68N_Wa_211215_0.2um_2 │ EMOBON00205 │          7.58 │
└─────────────────────────────────┴─────────────┴───────────────┘

In [11]:
duckdb.sql("SHOW TABLES")
# Abundant taxa found with LSU sequences that come from hot suface waters (> 20C) and their location

query = """
SELECT LSU.ncbi_taxid,
       LSU.abundance,
       SAMPLE_METADATA.sea_surf_temp,
       LSU.ref_code,
       OBS_METADATA.obs_id,
       OBS_METADATA.geo_loc_name,
       OBS_METADATA.loc_broad_ocean
FROM LSU
FULL JOIN SAMPLE_METADATA ON LSU.ref_code        = SAMPLE_METADATA.ref_code
FULL JOIN OBS_METADATA    ON OBS_METADATA.obs_id = SAMPLE_METADATA.obs_id
WHERE LSU.abundance > 1000.0 AND SAMPLE_METADATA.sea_surf_temp > 20.0
ORDER BY LSU.abundance DESC
"""
duckdb.sql(query).show(max_rows=107)

┌────────────┬───────────┬───────────────┬─────────────┬──────────┬──────────────┬───────────────────────────────────┐
│ ncbi_taxid │ abundance │ sea_surf_temp │  ref_code   │  obs_id  │ geo_loc_name │          loc_broad_ocean          │
│   int64    │   float   │    double     │   varchar   │ varchar  │   varchar    │              varchar              │
├────────────┼───────────┼───────────────┼─────────────┼──────────┼──────────────┼───────────────────────────────────┤
│      54526 │   14280.0 │      23.33631 │ EMOBON00125 │ IUIEilat │ Israel       │ Indian Ocean                      │
│      54526 │   12106.0 │      23.33631 │ EMOBON00124 │ IUIEilat │ Israel       │ Indian Ocean                      │
│      54526 │   11201.0 │       24.8503 │ EMOBON00121 │ IUIEilat │ Israel       │ Indian Ocean                      │
│      54526 │   10968.0 │       24.8503 │ EMOBON00120 │ IUIEilat │ Israel       │ Indian Ocean                      │
│      72037 │   10929.0 │      23.33631 │ EMOBO

# Summing abundances and pivoting taxonomy tables

see [Taxonomy Dataframes](./Taxonomy_dataframes.ipynb)

In [ ]:
query = """
SELECT class, sum(abundance) FROM LSU WHERE class = 'Thermoanaerobaculia' 
GROUP BY class
"""
duckdb.sql(query).show(max_width=250, max_rows=117)

In [92]:
query = """
SELECT
ref_code,
class,
sum(abundance) as total
FROM LSU 
WHERE class <> '' AND class IS NOT NULL
GROUP BY class, ref_code
ORDER BY class, sum(abundance) DESC
"""
duckdb.sql(query).show(max_width=10, max_rows=100)
r = duckdb.sql(query).to_df()

┌─────────────┬────────────────────┬────────┐
│  ref_code   │       class        │ total  │
│   varchar   │      varchar       │ double │
├─────────────┼────────────────────┼────────┤
│ EMOBON00142 │ Acidimicrobiia     │ 2130.0 │
│ EMOBON00140 │ Acidimicrobiia     │ 1843.0 │
│ EMOBON00143 │ Acidimicrobiia     │ 1633.0 │
│ EMOBON00195 │ Acidimicrobiia     │ 1494.0 │
│ EMOBON00139 │ Acidimicrobiia     │ 1434.0 │
│ EMOBON00085 │ Acidimicrobiia     │ 1186.0 │
│ EMOBON00138 │ Acidimicrobiia     │ 1174.0 │
│ EMOBON00194 │ Acidimicrobiia     │  963.0 │
│ EMOBON00125 │ Acidimicrobiia     │  946.0 │
│ EMOBON00242 │ Acidimicrobiia     │  940.0 │
│ EMOBON00133 │ Acidimicrobiia     │  926.0 │
│ EMOBON00124 │ Acidimicrobiia     │  864.0 │
│ EMOBON00136 │ Acidimicrobiia     │  850.0 │
│ EMOBON00141 │ Acidimicrobiia     │  763.0 │
│ EMOBON00236 │ Acidimicrobiia     │  760.0 │
│ EMOBON00094 │ Acidimicrobiia     │  692.0 │
│ EMOBON00093 │ Acidimicrobiia     │  683.0 │
│ EMOBON00243 │ Acidimicrobiia    

In [78]:
r.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5655 entries, 0 to 5654
Data columns (total 3 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   ref_code        5655 non-null   object 
 1   class           5655 non-null   object 
 2   sum(abundance)  5655 non-null   float64
dtypes: float64(1), object(2)
memory usage: 132.7+ KB


In [103]:
query = """
PIVOT r
ON ref_code
USING sum(total)
ORDER BY class
"""
duckdb.sql(query).show(max_width=30, max_rows=55)

┌──────────────────────┬─────────────┬───┬─────────────┬─────────────┐
│        class         │ EMOBON00001 │ … │ EMOBON00242 │ EMOBON00243 │
│       varchar        │   double    │   │   double    │   double    │
├──────────────────────┼─────────────┼───┼─────────────┼─────────────┤
│ Acidimicrobiia       │       493.0 │ … │       940.0 │       619.0 │
│ Acidithiobacillia    │         7.0 │ … │         8.0 │        10.0 │
│ Acidobacteriia       │        35.0 │ … │        87.0 │        56.0 │
│ Aconoidasida         │        NULL │ … │        NULL │        NULL │
│ Actinobacteria       │        51.0 │ … │        84.0 │        76.0 │
│ Actinopteri          │        NULL │ … │        NULL │        NULL │
│ Agaricomycetes       │        NULL │ … │         1.0 │        NULL │
│ Allomalorhagida      │         2.0 │ … │        NULL │        NULL │
│ Alphaproteobacteria  │      1432.0 │ … │      2933.0 │      2138.0 │
│ Anaerolineae         │        61.0 │ … │       134.0 │        91.0 │
│ Anth

##### 

In [104]:
query = """
PIVOT (
SELECT
ref_code,
class,
sum(abundance) as total
FROM LSU 
WHERE class <> '' AND class IS NOT NULL
GROUP BY class, ref_code
ORDER BY class, sum(abundance) DESC
)
ON ref_code
USING sum(total)
ORDER BY class
"""
duckdb.sql(query).show(max_width=30, max_rows=55)

┌──────────────────────┬─────────────┬───┬─────────────┬─────────────┐
│        class         │ EMOBON00001 │ … │ EMOBON00242 │ EMOBON00243 │
│       varchar        │   double    │   │   double    │   double    │
├──────────────────────┼─────────────┼───┼─────────────┼─────────────┤
│ Acidimicrobiia       │       493.0 │ … │       940.0 │       619.0 │
│ Acidithiobacillia    │         7.0 │ … │         8.0 │        10.0 │
│ Acidobacteriia       │        35.0 │ … │        87.0 │        56.0 │
│ Aconoidasida         │        NULL │ … │        NULL │        NULL │
│ Actinobacteria       │        51.0 │ … │        84.0 │        76.0 │
│ Actinopteri          │        NULL │ … │        NULL │        NULL │
│ Agaricomycetes       │        NULL │ … │         1.0 │        NULL │
│ Allomalorhagida      │         2.0 │ … │        NULL │        NULL │
│ Alphaproteobacteria  │      1432.0 │ … │      2933.0 │      2138.0 │
│ Anaerolineae         │        61.0 │ … │       134.0 │        91.0 │
│ Anth